In [1]:
from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
# from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

import np_utils
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import os

c:\Users\82105\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf
tf.__version__

'2.10.1'

In [36]:
current_path=os.getcwd().replace("\\", "/").replace("c:", "C:")
data_path=current_path+"/wavFiles/"
model_path=current_path+"/model/"

In [16]:
final_df=pd.read_csv(current_path+'/final_df.csv')

In [17]:
final_df.rename(columns={"filename":"wav_id", "label":"final_label"}, inplace=True)

In [18]:
final_df["final_label"].value_counts()

final_label
0    3872
1    1849
Name: count, dtype: int64

In [25]:
class text_embedding():
  def __init__(self, model_name):
    self.model_name = model_name

  def fit(self, X, y=None):
        return self

  def transform(self, X):
        embedding_model = SentenceTransformer(self.model_name)
        embedding_vec = embedding_model.encode(X['sentence'])
        X_val = np.concatenate((X.drop(['final_label', 'wav_id', 'sentence'], axis = 1), embedding_vec), axis = 1)
        return X_val

In [26]:
def custom_model(x_train):
  model = Sequential()
  model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
  model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

  model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

  model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
  model.add(Dropout(0.2))

  model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

  model.add(Flatten())
  model.add(Dense(units=32, activation='relu'))
  model.add(Dropout(0.3))

  model.add(Dense(units=2, activation='softmax'))  # 유닛 수를 2로 변경
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  # model.summary()
  return model


In [20]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=2, min_lr=0.0000001) #learning rate 조절
# 선택한 모델만 다시 학습 (텍스트 + 음성)
scaler = StandardScaler()
encoder = OneHotEncoder()

model_name='sentence-transformers/all-mpnet-base-v2'
Y = final_df['final_label'].values
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

txt_embed = text_embedding(model_name = model_name)
X = txt_embed.transform(final_df)

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [27]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

model = custom_model(x_train)
history=model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), callbacks=[rlrp])

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print("Pre-trained Model: ", model_name)
print("Test Accuracy: ",test_acc)

Epoch 1/50
68/68 [==============================] - 25s 351ms/step - loss: 0.6570 - accuracy: 0.6765 - val_loss: 0.6366 - val_accuracy: 0.6785 - lr: 0.0010
Epoch 2/50
68/68 [==============================] - 24s 352ms/step - loss: 0.6184 - accuracy: 0.6783 - val_loss: 0.5957 - val_accuracy: 0.6855 - lr: 0.0010
Epoch 3/50
68/68 [==============================] - 24s 353ms/step - loss: 0.5989 - accuracy: 0.6779 - val_loss: 0.5901 - val_accuracy: 0.6855 - lr: 0.0010
Epoch 4/50
68/68 [==============================] - 24s 352ms/step - loss: 0.5948 - accuracy: 0.6781 - val_loss: 0.5768 - val_accuracy: 0.6834 - lr: 0.0010
Epoch 5/50
68/68 [==============================] - 24s 351ms/step - loss: 0.5915 - accuracy: 0.6795 - val_loss: 0.5553 - val_accuracy: 0.6869 - lr: 0.0010
Epoch 6/50
68/68 [==============================] - 24s 359ms/step - loss: 0.5702 - accuracy: 0.6841 - val_loss: 0.5496 - val_accuracy: 0.7086 - lr: 0.0010
Epoch 7/50
68/68 [==============================] - 24s 351ms/st

In [50]:
with open(model_path+'text_model_name.txt', 'w') as file:
    file.write(model_name)
import joblib
joblib.dump(encoder, model_path+'encoder.pkl')
joblib.dump(scaler, model_path+'scaler.pkl')
model.save(model_path+'my_model.h5')

In [52]:
final_df

,0,1,2,3,4,5,6,7,8,9,...,479,480,481,482,483,484,485,wav_id,final_label,sentence
0,0.062374,0.648243,0.652445,0.659736,0.660877,0.671919,0.728619,0.662677,0.734502,0.715104,...,2.551432e-12,1.694380e-12,2.572616e-12,2.552685e-12,2.621055e-12,1.543373e-12,8.397591e-14,301_AUDIO_112268.0_113068.0_SPLIT.wav,0,the traffic
1,0.045654,0.617405,0.618014,0.647488,0.698651,0.712149,0.772312,0.760311,0.730769,0.720362,...,8.809110e-11,1.491751e-10,1.540081e-10,1.214503e-10,1.056656e-10,7.969472e-11,1.093357e-11,301_AUDIO_116188.0_122918.0_SPLIT.wav,0,the traffic is horrible well probably traffic ...
2,0.049818,0.401248,0.470091,0.554241,0.636612,0.687416,0.675771,0.566739,0.643228,0.749014,...,1.770267e-12,1.603301e-12,1.490871e-12,1.400250e-12,1.343897e-12,1.068107e-12,4.073818e-13,301_AUDIO_126698.0_135338.0_SPLIT.wav,0,not really i mean i have enough things going o...
3,0.035717,0.397980,0.516331,0.553759,0.498285,0.447431,0.525730,0.523040,0.498022,0.490343,...,1.100959e-11,1.001285e-11,9.346970e-12,8.793712e-12,8.499282e-12,6.772565e-12,2.681083e-12,301_AUDIO_142168.0_144338.0_SPLIT.wav,0,i studied uh business
4,0.024829,0.509619,0.641488,0.674360,0.636585,0.642636,0.653347,0.720315,0.857593,0.834892,...,5.147559e-11,4.896451e-11,4.906579e-11,4.753832e-11,5.560491e-11,3.415537e-11,2.674063e-12,301_AUDIO_145058.0_145518.0_SPLIT.wav,0,i did
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5716,0.067447,0.556869,0.679840,0.826999,0.704643,0.549325,0.477935,0.458215,0.536527,0.607850,...,1.714222e-13,1.561606e-13,1.418273e-13,1.188135e-13,1.235958e-13,9.520307e-14,2.676891e-14,483_AUDIO_945041.0_946101.0_SPLIT.wav,1,and i think
5717,0.130353,0.438964,0.599946,0.591212,0.594305,0.618439,0.603517,0.494303,0.549067,0.602864,...,5.178545e-12,4.016589e-12,3.778340e-12,5.103089e-12,5.359248e-12,4.127371e-12,5.739745e-13,483_AUDIO_948241.0_950831.0_SPLIT.wav,1,uh i i don't know i mean
5718,0.057671,0.391528,0.532826,0.593405,0.567645,0.558226,0.538988,0.468776,0.472341,0.546299,...,1.622510e-09,1.747478e-09,1.350160e-09,6.914879e-10,7.617159e-10,3.574834e-10,5.229890e-11,483_AUDIO_952132.0_956972.0_SPLIT.wav,1,you can try and look at you can try your best ...
5719,0.093669,0.648353,0.704458,0.538919,0.452962,0.438926,0.473726,0.551475,0.662206,0.695699,...,5.421763e-11,6.095318e-11,5.433036e-11,5.061723e-11,6.024754e-11,4.361564e-11,1.330648e-11,483_AUDIO_957982.0_960902.0_SPLIT.wav,1,in that but there's just there's nothing the only
